# Field Selection for Operations Rehearsal for Commissioning

In [ ]:
import numpy as np
import healpy as hp

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
m_stellar_density = np.load('/home/b/bechtol/rubin_sim_data/maps/StarMaps/starDensity_g_nside_64.npz')

In [ ]:
for key in m_stellar_density.keys():
    print(key)

In [ ]:
m_stellar_density['bins'][20]

In [ ]:
# Stars with g band magnitude of roughly 19
hp.mollview(np.log10(m_stellar_density['starDensity'][:,20]))
#hp.projscatter(ra_fields, dec_fields, lonlat=True, color='red')

In [ ]:
m_extinction = np.load('/home/b/bechtol/rubin_sim_data/maps/DustMaps/dust_nside_64.npz')

In [ ]:
hp.mollview(m_extinction['ebvMap'], min=0, max=0.1)

In [ ]:
selection = (m_extinction['ebvMap'] < 0.1) & (m_stellar_density['starDensity'][:,20] != hp.UNSEEN) & (m_stellar_density['starDensity'][:,20] > 0.)

median_stellar_density = np.median(m_stellar_density['starDensity'][:,20][selection])

plt.figure()
plt.hist(m_stellar_density['starDensity'][:,20][selection], bins=np.linspace(0, 2.e4, 51))
plt.axvline(median_stellar_density, c='black', ls='--')

## Suggested Fields

In [ ]:
# from rubin_scheduler.utils import ddf_locations
# ddf_locations = ddf_locations()

In [ ]:
# https://arxiv.org/abs/2306.06308
desi_targets = (
    (150.10, 2.182),
    (179.60, 0.000), 
    (183.10, 0.000), 
    (189.90, 61.800), 
    (194.75, 28.200), 
    (210.00, 5.000), 
    (215.50, 52.500), 
    (217.80, 34.400), 
    (216.30, -0.600), 
    (219.80, -0.600), 
    (218.05, 2.430), 
    (242.75, 54.980), 
    (241.05, 43.450), 
    (245.88, 43.450), 
    (252.50, 34.500), 
    (269.73, 66.020), 
    (194.75, 24.700), 
    (212.80, -0.600),
    (269.73, 62.520),
    (236.10, 43.450),
)
ra_desi, dec_desi = zip(*desi_targets)

In [ ]:
egal_targets = (
    (200, 0), 
    (15, -30),
    (0, 0),
    (54, -35),
    (345, -32),
    (35, -5),
    (35, -4),
    (350, -55),
    (61, -48),
    (67, -43),
    (10, -44),
    (53, -28),
    (150, 2),
    (201.365, -43.019), # Cen A
    (204.254, -29.865), # M83
    (11.888, -25.288), # NGC 253
)
ra_egal, dec_egal = zip(*egal_targets)

In [ ]:
# https://arxiv.org/abs/2310.19864
deep_targets = (
    (216, -12.5),
    (253, -21.5),
    (310, -19),
    (353, -5),
)
ra_deep, dec_deep = zip(*deep_targets)

In [ ]:
# https://www.cosmos.esa.int/web/euclid/euclid-survey
euclid_targets = (
    (269.73, 66.02),
    (52.93, -28.09),
    (61.42, -48.42),
)
ra_euclid, dec_euclid = zip(*euclid_targets)

In [ ]:
# 18:03:34.0 -30:02:02 1d01m28.358s -3d55m30.9507s DECam bulge field (Baade Window)
smwlv_tvs_targets = (
    (270.891667, -30.033889),
)

## Selected Fields for Operations Rehearsal Simulation

In [ ]:
import ops_rehearsal_fields

name_fields, ra_fields, dec_fields = zip(*ops_rehearsal_fields.fields)
for name, ra, dec in zip(name_fields, ra_fields, dec_fields):
    print(name, ra, dec)

In [ ]:
import copy
m = copy.deepcopy(m_stellar_density['starDensity'][:,20])

mask = (m_extinction['ebvMap'] > 0.1) | (m == 0)

m[mask] = hp.UNSEEN

#hp.mollview(np.log10(m))
hp.mollview(m, min=500, max=10000, unit='Stellar Density', title=None, coord='C')
hp.graticule()
#hp.projscatter(ra_fields, dec_fields, lonlat=True, color='red')
hp.projscatter(ra_fields, dec_fields, lonlat=True, color='white')
hp.projscatter(ra_desi, dec_desi, lonlat=True, facecolor='none', color='orange')
hp.projscatter(ra_egal, dec_egal, lonlat=True, facecolor='none', color='orange', marker='s')
hp.projscatter(ra_deep, dec_deep, lonlat=True, facecolor='none', color='red', marker='o')
hp.projscatter(ra_euclid, dec_euclid, lonlat=True, facecolor='none', color='red', marker='s')

In [ ]:
hp.mollview(m_extinction['ebvMap'], min=0, max=0.1, unit='E(B-V)', title=None, coord='C')
hp.graticule()
hp.projscatter(ra_fields, dec_fields, lonlat=True, color='white')

## Summary Stats for Selected Fields

In [ ]:
print("%20s%12s%12s%12s%12s"%(
    "Field",
    "RA",
    "dec",
    "stel dens",
    "E(B-V)"
))
nside=64
for name, ra, dec in zip(name_fields, ra_fields, dec_fields):
    pix = hp.ang2pix(nside, ra, dec, lonlat=True)
    stellar_density_ratio = m_stellar_density['starDensity'][:,20][pix] / median_stellar_density
    ebv = m_extinction['ebvMap'][pix]
    print(f"{name:20s}{ra:12.2f}{dec:12.2f}{stellar_density_ratio:12.2f}{ebv:12.2f}")